In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("ford_car_reviews.csv",nrows=25)

In [3]:
df.head()

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating
0,0,on 06/06/18 14:19 PM (PDT),Vicki,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,2006 Mustang GT,Doesn’t disappoint,5.0
1,1,on 08/12/17 06:06 AM (PDT),Tom,2006 Ford Mustang Coupe V6 Standard 2dr Coupe ...,DREAM CAR,I bought mine 4/17 with 98K. Have been wantin...,3.0
2,2,on 06/15/17 05:43 AM (PDT),Ray,2006 Ford Mustang Coupe V6 Premium 2dr Coupe (...,Great Ride,There will always be a 05-09 mustang for sale...,5.0
3,3,on 05/18/17 17:33 PM (PDT),Don Watson,2006 Ford Mustang Coupe V6 Deluxe 2dr Coupe (4...,I have wanted a Mustang for 40 years.,I bought my car from an auction I work at ( A...,5.0
4,4,on 01/03/16 18:03 PM (PST),One owner,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,One owner,I bought this car spankin new and i still am ...,5.0


In [4]:
df.drop("Unnamed: 0",axis=1,inplace=True)

In [5]:
df.head()

,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating
0,on 06/06/18 14:19 PM (PDT),Vicki,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,2006 Mustang GT,Doesn’t disappoint,5.0
1,on 08/12/17 06:06 AM (PDT),Tom,2006 Ford Mustang Coupe V6 Standard 2dr Coupe ...,DREAM CAR,I bought mine 4/17 with 98K. Have been wantin...,3.0
2,on 06/15/17 05:43 AM (PDT),Ray,2006 Ford Mustang Coupe V6 Premium 2dr Coupe (...,Great Ride,There will always be a 05-09 mustang for sale...,5.0
3,on 05/18/17 17:33 PM (PDT),Don Watson,2006 Ford Mustang Coupe V6 Deluxe 2dr Coupe (4...,I have wanted a Mustang for 40 years.,I bought my car from an auction I work at ( A...,5.0
4,on 01/03/16 18:03 PM (PST),One owner,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,One owner,I bought this car spankin new and i still am ...,5.0


In [8]:
import os, json, re, getpass
from dotenv import load_dotenv

load_dotenv(".env", override=True)
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ API Key: ")

GROQ API Key:  ········


In [9]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq

In [10]:
# Step 1: Initialize the model
model_id = "llama3-8b-8192"
llm = ChatGroq(model_name=model_id, temperature=0)

In [32]:
# Step 2: Create the ChatPromptTemplate with specific instructions
system_message = """
You are a sentiment analysis model. Classify the sentiment of the following review into one of the following categories:
- Positive
- Negative
- Neutral
Return only one of these words (Positive, Negative, Neutral) as the sentiment.
"""
human_message = "{review}"

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", human_message)
])

In [33]:
# Step 3: Create the LCEL Chain
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [34]:
# Step 4: Function to classify sentiment
def classify_sentiment(review):
    response = llm_chain.invoke({"review": review})
    print(response)  # Inspect the response to understand its structure
    sentiment = response.get('text', '').strip()  # Extract sentiment from response
    # Ensure only the three valid outputs are returned
    if sentiment.lower() not in ['positive', 'negative', 'neutral']:
        sentiment = 'Neutral'  # Default to 'Neutral' if unexpected response
    return sentiment

In [35]:
print(df.columns)

Index(['Review_Date', 'Author_Name', 'Vehicle_Title', 'Review_Title', 'Review',
       'Rating', 'sentiment'],
      dtype='object')


In [36]:
df['sentiment'] = df['Review'].apply(classify_sentiment)

{'review': ' Doesn’t disappoint', 'text': 'Neutral'}
{'review': ' I bought mine 4/17 with 98K. Have been wanting a V6 5-sp, \'05-\'09 vintage for years. The engine is fine. Sounds good. Great mileage. Good power. I pride myself on smooth take-off and gear changes, but this is the orneriest transmission I\'ve ever used! The difference between idle and 4000 rpm is about 1/8" on the gas pedal, so starting-out without either stalling or way over-revving takes a LOT of finesse. Gear changes are very difficult to master smoothly without lurching. The ride is very harsh with a lot of road noise, which I suppose goes with a quasi high-performance car. My \'01 S-10 is quieter and smoother. All that said, it\'s a smokin\' hot looking car, and still fun to drive. This was the first-off of the modern \'Stangs. Hopefully these problems have been ironed-out since.', 'text': 'Negative'}
{'review': ' There will always be a 05-09 mustang for sale and their fairly reasonable. Purchased mine as second ca

In [37]:
# Step 7: Check the resulting DataFrame
print(df[['Review', 'sentiment']])

                                               Review sentiment
0                                  Doesn’t disappoint   Neutral
1    I bought mine 4/17 with 98K. Have been wantin...  Negative
2    There will always be a 05-09 mustang for sale...  Positive
3    I bought my car from an auction I work at ( A...  Positive
4    I bought this car spankin new and i still am ...  Positive
5    Lots of problems with Ford these days, sensor...  Negative
6    Bought mine used 20k on it and have added a S...  Positive
7    I bought my preowned 06 a few weeks back and ...  Positive
8    I drive 50 miles each way to work and traded ...  Negative
9    This car is just awesome.  The 4.6L V8 makes ...  Positive
10   Only had car for one week and this car put th...  Positive
11   Previous mustang owner (1980) and have had a ...  Positive
12   I drive a lot of miles, at least 150/day.  I ...  Positive
13   I purchased this Horsey in memory of my dad, ...  Positive
14   Dash board rattles, dealer couldn't

In [38]:
df.to_csv('updated_file.csv', index=False)

In [39]:
df.head()

,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,sentiment
0,on 06/06/18 14:19 PM (PDT),Vicki,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,2006 Mustang GT,Doesn’t disappoint,5.0,Neutral
1,on 08/12/17 06:06 AM (PDT),Tom,2006 Ford Mustang Coupe V6 Standard 2dr Coupe ...,DREAM CAR,I bought mine 4/17 with 98K. Have been wantin...,3.0,Negative
2,on 06/15/17 05:43 AM (PDT),Ray,2006 Ford Mustang Coupe V6 Premium 2dr Coupe (...,Great Ride,There will always be a 05-09 mustang for sale...,5.0,Positive
3,on 05/18/17 17:33 PM (PDT),Don Watson,2006 Ford Mustang Coupe V6 Deluxe 2dr Coupe (4...,I have wanted a Mustang for 40 years.,I bought my car from an auction I work at ( A...,5.0,Positive
4,on 01/03/16 18:03 PM (PST),One owner,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,One owner,I bought this car spankin new and i still am ...,5.0,Positive
